# Подготовка данных

## Считаем данные в исходном csv формате

In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


Перезапустить kernel после установки!

In [2]:
import findspark
findspark.init()

In [3]:
!hdfs dfs -mkdir -p /user/ubuntu/data

In [4]:
!hadoop distcp s3a://otus-bucket-b1gcq77orp7o97v8lnkm/ /user/ubuntu/data

2025-12-03 18:05:47,905 INFO tools.DistCp: Input Options: DistCpOptions{atomicCommit=false, syncFolder=false, deleteMissing=false, ignoreFailures=false, overwrite=false, append=false, useDiff=false, useRdiff=false, fromSnapshot=null, toSnapshot=null, skipCRC=false, blocking=true, numListstatusThreads=0, maxMaps=20, mapBandwidth=0.0, copyStrategy='uniformsize', preserveStatus=[], atomicWorkPath=null, logPath=null, sourceFileListing=null, sourcePaths=[s3a://otus-bucket-b1gcq77orp7o97v8lnkm/], targetPath=/user/ubuntu/data, filtersFile='null', blocksPerChunk=0, copyBufferSize=8192, verboseLog=false, directWrite=false}, sourcePaths=[s3a://otus-bucket-b1gcq77orp7o97v8lnkm/], targetPathExists=true, preserveRawXattrsfalse
2025-12-03 18:05:48,049 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-12-03 18:05:48,168 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-12-03 18:05:48,169 INFO impl.MetricsSystemImpl: JobTracker metrics sy

In [5]:
!hdfs dfs -ls /user/ubuntu/data

Found 4 items
-rw-r--r--   1 ubuntu hadoop 3136657969 2025-12-03 18:06 /user/ubuntu/data/2022-11-04.txt
-rw-r--r--   1 ubuntu hadoop       9703 2025-12-03 18:05 /user/ubuntu/data/lectures.csv
-rw-r--r--   1 ubuntu hadoop     296161 2025-12-03 18:05 /user/ubuntu/data/questions.csv
-rw-r--r--   1 ubuntu hadoop   53700464 2025-12-03 18:05 /user/ubuntu/data/train_1000k.csv


In [6]:
!hdfs dfs -ls

Found 1 items
drwxr-xr-x   - ubuntu hadoop          0 2025-12-03 18:06 data


In [7]:
!hdfs dfs -ls data

Found 4 items
-rw-r--r--   1 ubuntu hadoop 3136657969 2025-12-03 18:06 data/2022-11-04.txt
-rw-r--r--   1 ubuntu hadoop       9703 2025-12-03 18:05 data/lectures.csv
-rw-r--r--   1 ubuntu hadoop     296161 2025-12-03 18:05 data/questions.csv
-rw-r--r--   1 ubuntu hadoop   53700464 2025-12-03 18:05 data/train_1000k.csv


Создадим SparkSession

In [8]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

Загрузим данные

In [9]:
df = spark.read.csv("data/train_1000k.csv", inferSchema=True, header=True)

Поделим данные на 10 разделов и сохраним в формате parquet

In [18]:
%%time
df.count()

CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 626 ms


1000000

In [12]:
(
    df
        .repartition(10)
        .write
        .mode("overwrite")
        .parquet("data/train.parquet")
)

Проверим размер сохраненного файла

In [13]:
!hdfs dfs -ls -h data/train.parquet

Found 11 items
-rw-r--r--   1 ubuntu hadoop          0 2025-12-03 18:08 data/train.parquet/_SUCCESS
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00000-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00001-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00002-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00003-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00004-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.8 M 2025-12-03 18:08 data/train.parquet/part-00005-09842bc8-6262-4feb-a900-789d9cb1e6df-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop      1.

In [14]:
df_from_parquet = spark.read.parquet("data/train.parquet")

In [19]:
%%time
df_from_parquet.count()

CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 303 ms


1000000

In [25]:
df_from_parquet.show(5)

+------+-----------+--------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|  timestamp| user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+-----------+--------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
| 62896|   96541616| 1400354|      1087|              0|               94|          3|                 0|                    17000.0|                         false|
|104413|   21162902| 2211492|      5262|              0|               95|          0|                 1|                    11000.0|                          true|
|537225|    4538020|10854346|      7880|              0|               26|          1|                 1|                    30000.0|                          true|
|158202|  